# Exemple de démarrage envoyé par Pascal, basé sur la lib statsmodels

L'enjeu, c'est la capacité à décider de la sélection de caractéristiques dès le stade de la régression linéaire. Comment distinguer les couples (coefficient, variable) pertinents de ceux qui ne le sont pas tout en contribuent au bruit et au coût d'opérations.

Article de commentaire : [Interpreting Linear Regression Through statsmodels .summary()](https://medium.com/swlh/interpreting-linear-regression-through-statsmodels-summary-4796d359035a)

Source primaire commentée : [StatsModels USer Guide | Fitting models using R-style formulas](https://www.statsmodels.org/dev/example_formulas.html)

## Ajustement de modèles à l'aide de formules de style R

Depuis la version 0.5.0, `statsmodels` permet aux utilisateurs d'ajuster des modèles statistiques à l'aide de formules de style R. En interne, `statsmodels` utilise le package [`patsy`](https://patsy.readthedocs.io/en/latest/) pour convertir les formules et les données en matrices utilisées dans l'ajustement du modèle. Le cadre de formule est assez puissant ; ce tutoriel ne fait qu'effleurer la surface. Une description complète du langage de formule peut être trouvée dans les docs patsy :

[Description du langage de formules Patsy](https://patsy.readthedocs.io/en/latest/)

### Chargement des modules et des fonctions

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas

Notez que nous avons appelé `statsmodels.formula.api` en plus du `statsmodels.api` habituel. En fait, `statsmodels.api` est utilisé ici uniquement pour charger le jeu de données. Le `formula.api` héberge bon nombre des mêmes fonctions trouvées dans `api` (par exemple OLS, GLM), mais il contient également des homologues en minuscules pour la plupart de ces modèles. En général, les modèles en minuscules acceptent les arguments `formula` et `df`, tandis que les majuscules acceptent les matrices de conception `endog` et `exog`. `formula` accepte une chaîne qui décrit le modèle en termes de formule `patsy`. df prend une trame de données `pandas`.

`dir(smf)` imprimera une liste des modèles disponibles.

Les modèles compatibles avec les formules ont la signature d'appel générique suivante : `(formula, data, subset=None, *args, **kwargs)`

### Régression OLS à l'aide de formules

Pour commencer, nous adaptons le modèle linéaire décrit sur la page [Mise en route](https://www.statsmodels.org/dev/gettingstarted.html). Téléchargez les données, les colonnes de sous-ensemble et la suppression par liste pour supprimer les observations manquantes :

In [ ]:
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
df.head()

,Lottery,Literacy,Wealth,Region
0,41,37,73,E
1,38,51,22,N
2,66,13,61,C
3,80,46,76,E
4,79,69,83,E


Ajuster le modèle :

In [ ]:
mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     6.636
Date:                Thu, 03 Nov 2022   Prob (F-statistic):           1.07e-05
Time:                        11:41:06   Log-Likelihood:                -375.30
No. Observations:                  85   AIC:                             764.6
Df Residuals:                      78   BIC:                             781.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      38.6517      9.456      4.087      

#### Commentaire

Ne soyez pas intimidé par les grands mots et les chiffres ! Ce blog est là pour traduire toutes ces informations en anglais simple. Notre objectif est de fournir un aperçu général de toutes les statistiques. Des recherches supplémentaires sont fortement recommandées pour une analyse approfondie de chaque composant.
Commençons par le début.

La première ligne de code qui nous manque ici est `import statsmodels.formula.api as smf`. Donc, ce que nous faisons ici, c'est d'utiliser la fonction fournie `ols()` ou Ordinary Least Squares de la bibliothèque statsmodels. OLS est une technique couramment utilisée pour analyser la régression linéaire. En bref, il compare la différence entre les points individuels de votre ensemble de données et la ligne de meilleur ajustement prévue pour mesurer la quantité d'erreur produite. La fonction `smf.ols()` nécessite deux entrées, la formule pour produire la meilleure ligne d'ajustement et l'ensemble de données.

La formule est fournie sous forme de chaîne, sous la forme suivante : 'variable dépendante ~ liste de variables indépendantes séparées par le symbole +' En clair, la variable dépendante est le facteur que vous essayez de prédire, et de l'autre côté de la formule sont les variables que vous utilisez pour prédire. L'ensemble de données dans ce cas est nommé 'df' et est utilisé pour déterminer le pari par habitant à la loterie royale de la France des années 1830 en utilisant quelques caractéristiques. Aux fins de cette leçon, les données ne sont pas pertinentes mais sont disponibles https://cran.r-project.org/web/packages/HistData/HistData.pdf pour votre intérêt.

Notre première ligne de code crée un modèle, nous l'appelons donc 'mod' et la seconde utilise le modèle pour créer une ligne de meilleur ajustement, d'où la régression linéaire. Nous l'appelons 'res' car il analyse les résidus de notre modèle. Ensuite, nous imprimons notre résumé.

    OLS Regression Results                            
    ==============================================================================
    Dep. Variable:                Lottery   R-squared:                       0.338
    Model:                            OLS   Adj. R-squared:                  0.287
    Method:                 Least Squares   F-statistic:                     6.636
    Date:                Thu, 03 Nov 2022   Prob (F-statistic):           1.07e-05
    Time:                        11:41:06   Log-Likelihood:                -375.30
    No. Observations:                  85   AIC:                             764.6
    Df Residuals:                      78   BIC:                             781.7
    Df Model:                           6                                         
    Covariance Type:            nonrobust
                                         

Le haut de notre résumé commence par nous donner quelques détails que nous connaissons déjà. Notre **variable dépendante** est la « loterie », nous utilisons les OLS connus sous le nom de moindres carrés ordinaires, ainsi que la **date** et l'**heure** auxquelles nous avons créé le **modèle**. Ensuite, il détaille notre **nombre d'observations** dans l'ensemble de données. **Df Residuals** est un autre nom pour nos degrés de liberté dans notre mode. Ceci est calculé sous la forme de $n-k-1$ ou "nombre d'observations-nombre de variables prédictives-1". **Df model** numérote nos variables prédictives. Si vous vous demandez pourquoi nous n'avons entré que 3 variables prédictives dans la formule alors que Df Residuals et Model disent qu'il y en a 6, nous y reviendrons plus tard. Notre **type de covariance** est répertorié comme non robuste. La covariance est une mesure de la manière dont deux variables sont liées de manière positive ou négative, et une covariance robuste est celle qui est calculée de manière à minimiser ou éliminer les variables, ce qui n'est pas le cas ici.

Le **R-squared** ($R^2$) est peut-être la mesure la plus importante produite par ce résumé. Le $R^2$ est la mesure de la part de la variable indépendante expliquée par les changements de nos variables dépendantes. En termes de pourcentage, 0,338 signifierait que notre modèle explique 33,8 % de la variation de notre variable "Loterie". Le **R-carré ajusté** est important pour analyser l'efficacité de plusieurs variables dépendantes sur le modèle. La régression linéaire a la qualité que la valeur R au carré de votre modèle ne diminuera jamais avec des variables supplémentaires, seulement égales ou supérieures. Par conséquent, votre modèle pourrait sembler plus précis avec plusieurs variables, même si elles contribuent peu. Le R-carré ajusté pénalise la formule R-carré basée sur le nombre de variables, donc un score ajusté inférieur peut vous indiquer que certaines variables ne contribuent pas correctement au R-carré de votre modèle.

La **statistique F** (*F-statistic*) en régression linéaire compare votre modèle linéaire produit pour vos variables à un modèle qui remplace l'effet de vos variables par 0, pour savoir si votre groupe de variables est statistiquement significatif. Pour interpréter correctement ce nombre, il est nécessaire d'utiliser une valeur alpha choisie et une **table F**. **Prob (F-Statistic)** utilise ce nombre pour vous indiquer la précision de l'hypothèse nulle, ou s'il est exact que l'effet de vos variables est de 0. Dans ce cas, il nous indique 0,00107 % de chance. La **log-vraisemblance** est un signifiant numérique de la probabilité que votre modèle produit produise ces données. Il est utilisé pour comparer les valeurs des coefficients de chaque variable lors du processus de création du modèle. **AIC** et **BIC** sont tous deux utilisés pour comparer l'efficacité des modèles dans le processus de régression linéaire, en utilisant un système de pénalité pour mesurer plusieurs variables. Ces nombres sont utilisés pour la sélection de caractéristiques sur l'ensemble des variables.

    ===============================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
    -------------------------------------------------------------------------------
    Intercept      38.6517      9.456      4.087      0.000      19.826      57.478
    Region[T.E]   -15.4278      9.727     -1.586      0.117     -34.793       3.938
    Region[T.N]   -10.0170      9.260     -1.082      0.283     -28.453       8.419
    Region[T.S]    -4.5483      7.279     -0.625      0.534     -19.039       9.943
    Region[T.W]   -10.0913      7.196     -1.402      0.165     -24.418       4.235
    Literacy       -0.1858      0.210     -0.886      0.378      -0.603       0.232
    Wealth          0.4515      0.103      4.390      0.000       0.247       0.656
    ==============================================================================
    Omnibus:                        3.049   Durbin-Watson:                   1.785
    Prob(Omnibus):                  0.218   Jarque-Bera (JB):                2.694
    Skew:                          -0.340   Prob(JB):                        0.260
    Kurtosis:                       2.454   Cond. No.                         371.
    ==============================================================================

Voyons maintenant le travail de notre modèle ! Décomposons-le.

L'interception est le résultat de notre modèle si toutes les variables étaient réglées sur 0. Dans la formule linéaire classique $y = mx + b$, c'est notre $b$, une constante ajoutée pour expliquer une valeur de départ pour notre ligne.

Sous l'interception se trouvent nos variables. Vous souvenez-vous de notre formule ? « Loterie ~ Région + Alphabétisation + Richesse » Ici, nous voyons nos variables dépendantes représentées. Mais pourquoi y a-t-il quatre versions différentes de Region alors que nous n'en saisissons qu'une ? En termes simples, la formule attend des valeurs continues sous forme de nombres. En saisissant une région avec des points de données sous forme de chaînes, la formule sépare chaque chaîne en catégories et analyse la catégorie séparément. Formater vos données à l'avance peut vous aider à les organiser et à les analyser correctement.

Notre première colonne informative est le coefficient. Pour notre interception, c'est la valeur de l'interception. Pour chaque variable, c'est la mesure de la façon dont le changement de cette variable affecte la variable indépendante. C'est le $m$ dans $y = mx + b$. Une unité de changement dans la variable dépendante affectera la valeur du coefficient de changement de la variable dans la variable indépendante. Si le coefficient est négatif, ils ont une relation inverse. Quand l'un monte, l'autre tombe.

Notre erreur standard est une estimation de l'écart type du coefficient, une mesure de la quantité de variation du coefficient à travers ses points de données. Le $t$ est lié et est une mesure de la précision avec laquelle le coefficient a été mesuré. Une erreur std faible par rapport à un coefficient élevé produit une statistique t élevée, ce qui signifie une signification élevée pour votre coefficient.

`P>|t|` est l'une des statistiques les plus importantes du résumé. Il utilise la statistique `t` pour produire la *valeur p*, une mesure de la probabilité que votre coefficient soit mesuré par notre modèle par hasard. La valeur p de 0,378 pour Richesse indique qu'il y a 37,8 % de chances que la variable Richesse n'ait aucun effet sur la variable dépendante, Loterie, et nos résultats sont produits par hasard. Une analyse de modèle appropriée comparera la valeur p à une valeur alpha précédemment établie, ou à un seuil avec lequel nous pouvons appliquer une signification à notre coefficient. Un alpha commun est de 0,05, que peu de nos variables réussissent dans ce cas.

`[0,025` et `0,975]` sont tous deux des mesures des valeurs de nos coefficients à moins de 95 % de nos données, ou à moins de deux écarts-types. En dehors de ces valeurs peuvent généralement être considérés comme des valeurs aberrantes.

**Omnibus** décrit la normalité de la distribution de nos résidus en utilisant le biais (skew) et l'applatissement (kurtosis) comme mesures. Un 0 indiquerait une normalité parfaite. **Prob(Omnibus)** est un test statistique mesurant la probabilité que les résidus soient normalement distribués. Un 1 indiquerait une distribution parfaitement normale. Skew est une mesure de symétrie dans nos données, 0 étant une symétrie parfaite. L'aplatissement mesure le pic de nos données, ou sa concentration autour de 0 dans une courbe normale. Un aplatissement plus élevé implique moins de valeurs aberrantes.

**Durbin-Watson** est une mesure de l'homoscédasticité, ou une distribution uniforme des erreurs dans nos données. L'hétéroscédasticité impliquerait une distribution inégale, par exemple, à mesure que le point de données augmente, l'erreur relative augmente. L'homoscédasticité idéale se situera entre 1 et 2. **Jarque-Bera (JB)** et **Prob(JB)** sont des méthodes alternatives de mesure de la même valeur que Omnibus et Prob(Omnibus) en utilisant l'asymétrie et l'aplatissement. Nous utilisons ces valeurs pour nous confirmer mutuellement. Le **nombre de conditions** est une mesure de la sensibilité de notre modèle par rapport à la taille des changements dans les données qu'il analyse. La multicolinéarité est fortement impliquée par un nombre de conditions élevé. Multicolinéarité un terme pour décrire deux ou plusieurs variables indépendantes qui sont fortement liées les unes aux autres et affectent faussement notre variable prédite par redondance.

Nos définitions effleurent à peine la surface de l'un ou l'autre de ces sujets. La recherche indépendante est fortement encouragée pour comprendre ces termes et leurs relations les uns avec les autres. J'espère que ce blog vous a donné suffisamment de compréhension pour commencer à interpréter votre modèle et les moyens de l'améliorer !

### Variables catégorielles

En regardant le résumé imprimé ci-dessus, notez que `patsy` a déterminé que les éléments de *Region* étaient des chaînes de texte, il a donc traité *Region* comme une variable catégorielle. La valeur par défaut de `patsy` est également d'inclure une interception, nous avons donc automatiquement supprimé l'une des catégories *Région*.

Si *Region* avait été une variable entière que nous voulions traiter explicitement comme catégorielle, nous aurions pu le faire en utilisant l'opérateur `C()` :

In [ ]:
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region)', data=df).fit()
print(res.params)

Intercept         38.651655
C(Region)[T.E]   -15.427785
C(Region)[T.N]   -10.016961
C(Region)[T.S]    -4.548257
C(Region)[T.W]   -10.091276
Literacy          -0.185819
Wealth             0.451475
dtype: float64


Des exemples de fonctions `patsy` plus avancées de variables catégorielles de caractéristiques peuvent être trouvés ici : [Patsy : systèmes de codage de contraste pour les variables catégorielles](https://www.statsmodels.org/dev/contrasts.html).

### Opérateurs

Nous avons déjà vu que "~" sépare le côté gauche du modèle du côté droit, et que "+" ajoute de nouvelles colonnes à la matrice de conception.

#### Suppression de variables

Le signe "-" peut être utilisé pour supprimer des colonnes/variables. Par exemple, nous pouvons supprimer l'interception d'un modèle en :

In [ ]:
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) -1 ', data=df).fit()
print(res.params)

C(Region)[C]    38.651655
C(Region)[E]    23.223870
C(Region)[N]    28.634694
C(Region)[S]    34.103399
C(Region)[W]    28.560379
Literacy        -0.185819
Wealth           0.451475
dtype: float64


#### Interactions multiplicatives

":" ajoute une nouvelle colonne à la matrice de conception avec le produit des deux autres colonnes. "*" inclura également les colonnes individuelles qui ont été multipliées ensemble :

In [ ]:
res1 = smf.ols(formula='Lottery ~ Literacy : Wealth - 1', data=df).fit()
res2 = smf.ols(formula='Lottery ~ Literacy * Wealth - 1', data=df).fit()
print(res1.params)
print(res2.params)

Literacy:Wealth    0.018176
dtype: float64
Literacy           0.427386
Wealth             1.080987
Literacy:Wealth   -0.013609
dtype: float64


Beaucoup d'autres choses sont possibles avec les opérateurs. Veuillez consulter la [documentation `patsy`](https://patsy.readthedocs.io/en/latest/formulas.html) pour en savoir plus.

### Fonctions

Vous pouvez appliquer des fonctions vectorisées aux variables de votre modèle :

In [ ]:
res = smf.ols(formula='Lottery ~ np.log(Literacy)', data=df).fit()
print(res.params)

Intercept           115.609119
np.log(Literacy)    -20.393959
dtype: float64


Définissez une fonction personnalisée :

In [ ]:
def log_plus_1(x):
    return np.log(x) + 1.0

res = smf.ols(formula='Lottery ~ log_plus_1(Literacy)', data=df).fit()
print(res.params)

Intercept               136.003079
log_plus_1(Literacy)    -20.393959
dtype: float64


### Espaces de noms

Notez que tous les exemples ci-dessus utilisent l'espace de noms appelant pour rechercher les fonctions à appliquer. L'espace de noms utilisé peut être contrôlé via le mot-clé `eval_env`. Par exemple, vous pouvez donner un espace de noms personnalisé en utilisant **`patsy:patsy.EvalEnvironment`** ou vous pouvez utiliser un espace de noms "propre", que nous fournissons en passant `eval_func=-1`. La valeur par défaut consiste à utiliser l'espace de noms de l'appelant. Cela peut avoir des conséquences (non) attendues, si, par exemple, quelqu'un a un nom de variable `C` dans l'espace de noms d'utilisateur ou dans sa structure de données transmise à `patsy`, et `C` est utilisé dans la formule pour gérer une variable catégorielle. Consultez la [référence de l'API Patsy](https://patsy.readthedocs.io/en/latest/API-reference.html) pour plus d'informations.

### Utiliser des formules avec des modèles qui ne les supportent pas (encore)

Même si une fonction `statsmodels` donnée ne prend pas en charge les formules, vous pouvez toujours utiliser le langage de formule de `patsy` pour produire des matrices de conception. Ces matrices peuvent ensuite être transmises à la fonction d'ajustement en tant qu'arguments `endog` et `exog`.

Pour générer des tableaux `numpy` :

In [ ]:
import patsy
f = 'Lottery ~ Literacy * Wealth'
y, X = patsy.dmatrices(f, df, return_type='matrix')
print(y[:5])
print(X[:5])

[[41.]
 [38.]
 [66.]
 [80.]
 [79.]]
[[1.000e+00 3.700e+01 7.300e+01 2.701e+03]
 [1.000e+00 5.100e+01 2.200e+01 1.122e+03]
 [1.000e+00 1.300e+01 6.100e+01 7.930e+02]
 [1.000e+00 4.600e+01 7.600e+01 3.496e+03]
 [1.000e+00 6.900e+01 8.300e+01 5.727e+03]]


In [ ]:
print(sm.OLS(y, X).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                Lottery   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     12.06
Date:                Thu, 03 Nov 2022   Prob (F-statistic):           1.32e-06
Time:                        12:10:22   Log-Likelihood:                -377.13
No. Observations:                  85   AIC:                             762.3
Df Residuals:                      81   BIC:                             772.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          38.6348     15.825     